In [6]:
from glob import glob
import afidutils
import pandas as pd
import numpy as np
from scipy.stats import mannwhitneyu  
from scipy.stats import ttest_ind


In [2]:
stn_gt = sorted(glob('//Users/alaataha/Documents/GitHub/afids-pred/data/coordinate/STN/*/stn_mcp/*/anat/*space-MCPPMJ_desc-groundtruth_stn.fcsv'))
stn_vols = "/Users/alaataha/Documents/GitHub/afids-pred/data/coordinate/STN/stn_native/STN_all.csv"
stn_vol_df = pd.read_csv(stn_vols)
print(f'Number of STN annotations: {len(stn_gt)}')

Number of STN annotations: 70


In [3]:
# Initialize containers for subject IDs, predictions, and ground truths
groundtruths = []
dataset = [] 
disease = []
hemi = []
vol = []

# Iterate over each subject ID in the test dataset
for subject in stn_gt:
    # Determine STN ground truth file path based on scanner metadata
    meta_data = afidutils.extract_sub_metadata(subject)
    scanner_type = meta_data[-2]
    subjid = meta_data[0]
    dataset.extend([scanner_type,scanner_type])
    if scanner_type == 'SNSXPD':
        disease.extend(['PD','PD'])
    else: 
        disease.extend(['HC','HC'])
    # Load and process STN ground truth coordinates
    df_stn = afidutils.fcsvtodf(subject)[0]
    df_stn.columns = ['RSTNx', 'LSTNx', 'RSTNy', 'LSTNy', 'RSTNz', 'LSTNz']

    # Mirror left STN x-coordinates for alignment
    df_stn['LSTNx'] *= -1

    # Reformat right and left STN coordinates to common schema
    df_stn_r = df_stn[['RSTNx', 'RSTNy', 'RSTNz']].rename(columns=lambda c: c[1:])
    df_stn_l = df_stn[['LSTNx', 'LSTNy', 'LSTNz']].rename(columns=lambda c: c[1:])
    df_stn_proc = pd.concat([df_stn_r, df_stn_l], axis=0)
    
    hemi.extend(['R','L'])
    groundtruths.append(df_stn_proc)

    filtered_df_R = stn_vol_df[(stn_vol_df["subjid"] == subjid) & (stn_vol_df["side"] == 'R')]['vol_mm'].values[0]
    filtered_df_L = stn_vol_df[(stn_vol_df["subjid"] == subjid) & (stn_vol_df["side"] == 'L')]['vol_mm'].values[0]
    vol.extend([filtered_df_R,filtered_df_L])

gt_coords = pd.concat(groundtruths, axis=0) 
gt_coords['dataset'] = dataset
gt_coords['disease'] = disease
gt_coords['side'] = hemi
gt_coords['vol'] = vol

In [10]:
# Load the CSV file into a DataFrame
df = gt_coords

# Since the CSV already has only the STN data and includes a 'disease' column,
# there's no need to filter for a specific ROI.

# Calculate summary statistics for STN volume (vol_mm) by group
vol_stats = df.groupby('disease')['vol'].agg(['mean', 'std'])
print("STN Volume Statistics (mm³):")
print(vol_stats)

# Calculate the average MCP coordinates (centroid_x, centroid_y, centroid_z) by group
coords_stats = df.groupby('disease')[['STNx', 'STNy', 'STNz']].mean()
print("\nAverage MCP Coordinates:")
print(coords_stats)

# List of variables to compare between HC and PD groups
variables = ['vol', 'STNx', 'STNy', 'STNz']
p_values = {}


# For each variable, perform a two-sided Mann–Whitney U test (non-parametric test)
for var in variables:
    hc_values = df[df['disease'] == 'HC'][var]
    pd_values = df[df['disease'] == 'PD'][var]
    stat, p = mannwhitneyu(hc_values, pd_values, alternative='two-sided')
    p_values[var] = p
    print(f"\nMann–Whitney U test for {var}: statistic = {stat:.3f}, p-value = {p:.5f}")
    
# Apply Bonferroni correction for multiple comparisons (4 tests)
bonferroni_corrected = {var: min(p * len(variables), 1.0) for var, p in p_values.items()}
print("\nBonferroni Corrected p-values:")
for var, cp in bonferroni_corrected.items():
    print(f"{var}: {cp:.5f}")

# Extract summary statistics for reporting
hc_vol_mean = vol_stats.loc['HC', 'mean']
hc_vol_std  = vol_stats.loc['HC', 'std']
pd_vol_mean = vol_stats.loc['PD', 'mean']
pd_vol_std  = vol_stats.loc['PD', 'std']

hc_coords = coords_stats.loc['HC']
pd_coords = coords_stats.loc['PD']

# Construct and print the summary report
print("\n--- Report Summary ---")
print(f"The average volume of the STN for the healthy control (HC) and Parkinson’s disease (PD) "
      f"cohorts was: {hc_vol_mean:.2f} ± {hc_vol_std:.2f} mm³ and {pd_vol_mean:.2f} ± {pd_vol_std:.2f} mm³, respectively.")
print(f"The average MCP coordinates across HC and PD cohorts were: "
      f"{hc_coords['STNx']:.2f}, {hc_coords['STNy']:.2f}, {hc_coords['STNz']:.2f} and "
      f"{pd_coords['STNx']:.2f}, {pd_coords['STNy']:.2f}, {pd_coords['STNz']:.2f}, respectively.")

# Determine if there is no statistical difference after Bonferroni correction
if all(cp > 0.05 for cp in bonferroni_corrected.values()):
    print("\nThere was no statistical difference between STN volume and MCP (x, y, z) coordinates across HC and PD "
          "(p > 0.05; Wilcoxon ranked test with Bonferroni correction).")
else:
    print("\nOne or more comparisons showed a statistically significant difference after Bonferroni correction.")


STN Volume Statistics (mm³):
               mean        std
disease                       
HC       136.558220  16.235342
PD       128.034859  29.676967

Average MCP Coordinates:
              STNx      STNy      STNz
disease                               
HC       10.178443 -0.916612 -3.272723
PD       10.135173 -0.877898 -3.609443

Mann–Whitney U test for vol: statistic = 2818.000, p-value = 0.04770

Mann–Whitney U test for STNx: statistic = 2373.000, p-value = 0.93051

Mann–Whitney U test for STNy: statistic = 2324.000, p-value = 0.90688

Mann–Whitney U test for STNz: statistic = 3065.000, p-value = 0.00244

Bonferroni Corrected p-values:
vol: 0.19080
STNx: 1.00000
STNy: 1.00000
STNz: 0.00976

--- Report Summary ---
The average volume of the STN for the healthy control (HC) and Parkinson’s disease (PD) cohorts was: 136.56 ± 16.24 mm³ and 128.03 ± 29.68 mm³, respectively.
The average MCP coordinates across HC and PD cohorts were: 10.18, -0.92, -3.27 and 10.14, -0.88, -3.61, respectiv